## Indice
#### 1. Pulizia crawled data
#### 2.Creo una colonna con i tag puliti
#### 3.Creazione network di partenza
#### 4.Creazione dataset da 10k nodi
###### 4.1 Controllo preliminare dei nodi ed edges
#### 5. Risultati

In [1]:
import twint
import pandas as pd


import networkx as nx
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import powerlaw
warnings.filterwarnings('ignore')


%matplotlib inline 

### PULIZIA CRAWLED DATA

In [2]:
df = pd.read_csv("../Dataset FFF/climatechange.csv", sep='\t')

In [3]:
type(df)

pandas.core.frame.DataFrame

In [4]:
#riduco le dimensioni del dataframe per avvicinarcio ai 15k, lo faccio togliendo parte finale dei dati 
df = df.iloc[:41700]

In [5]:
df.tail(2)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
41698,1187693451556610049,1187693451556610049,2019-10-25 13:32:39 ora legale Europa occidentale,2019-10-25,13:32:39,200,712989553,just_aenna,Anna Hübner,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
41699,1187693166742429696,1187693166742429696,2019-10-25 13:31:31 ora legale Europa occidentale,2019-10-25,13:31:31,200,68459659,winterbe_,Benjamin,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [6]:
# Get all names 
for col_name in df.columns: 
    print(col_name)

id
conversation_id
created_at
date
time
timezone
user_id
username
name
place
tweet
language
mentions
urls
photos
replies_count
retweets_count
likes_count
hashtags
cashtags
link
retweet
quote_url
video
thumbnail
near
geo
source
user_rt_id
user_rt
retweet_id
reply_to
retweet_date
translate
trans_src
trans_dest


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41700 entries, 0 to 41699
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               41700 non-null  int64  
 1   conversation_id  41700 non-null  int64  
 2   created_at       41700 non-null  object 
 3   date             41700 non-null  object 
 4   time             41700 non-null  object 
 5   timezone         41700 non-null  int64  
 6   user_id          41700 non-null  int64  
 7   username         41700 non-null  object 
 8   name             41697 non-null  object 
 9   place            12 non-null     object 
 10  tweet            41700 non-null  object 
 11  language         41700 non-null  object 
 12  mentions         41700 non-null  object 
 13  urls             41700 non-null  object 
 14  photos           41700 non-null  object 
 15  replies_count    41700 non-null  int64  
 16  retweets_count   41700 non-null  int64  
 17  likes_count 

Toglie created_at, time, timezone, lasciando la data
Sono dati ripetuti 

In [8]:
df.drop(['trans_dest', 'trans_src','translate','retweet_date','geo','source','user_rt_id', 'user_rt','retweet_id',
        'near','place','thumbnail', 'time', 'timezone', 'video', 'photos', 'cashtags', 'created_at','quote_url','id','conversation_id'], axis=1, inplace=True)





In [9]:
#No valori nulli!
df['language'].isnull().values.any()

False

In [10]:
Lingue=list(set(df['language']))
#Lingue

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41700 entries, 0 to 41699
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   date            41700 non-null  object
 1   user_id         41700 non-null  int64 
 2   username        41700 non-null  object
 3   name            41697 non-null  object
 4   tweet           41700 non-null  object
 5   language        41700 non-null  object
 6   mentions        41700 non-null  object
 7   urls            41700 non-null  object
 8   replies_count   41700 non-null  int64 
 9   retweets_count  41700 non-null  int64 
 10  likes_count     41700 non-null  int64 
 11  hashtags        41700 non-null  object
 12  link            41700 non-null  object
 13  retweet         41700 non-null  bool  
 14  reply_to        41700 non-null  object
dtypes: bool(1), int64(4), object(10)
memory usage: 4.5+ MB


Restingendo il range di tweet con numero di likes, retweet e replies
Si arriva a 34848 valori totali

In [12]:
df=df[df['likes_count']>=6]
df=df[df['replies_count']>=1]
df=df[df['retweets_count']>=3]

In [13]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28381 entries, 5 to 41699
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   date            28381 non-null  object
 1   user_id         28381 non-null  int64 
 2   username        28381 non-null  object
 3   name            28380 non-null  object
 4   tweet           28381 non-null  object
 5   language        28381 non-null  object
 6   mentions        28381 non-null  object
 7   urls            28381 non-null  object
 8   replies_count   28381 non-null  int64 
 9   retweets_count  28381 non-null  int64 
 10  likes_count     28381 non-null  int64 
 11  hashtags        28381 non-null  object
 12  link            28381 non-null  object
 13  retweet         28381 non-null  bool  
 14  reply_to        28381 non-null  object
dtypes: bool(1), int64(4), object(10)
memory usage: 3.3+ MB


### Creo una colonna con i tag puliti

In [14]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marco\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Faccio questa funzione per pulire i tags dalle quadre, la applico a una nuova colonna (clean_tags)

In [15]:
def remove_content(text):
    text = re.sub(r"\[", "", text) #remove ]
    text=re.sub(r'\]','',text) #remove ]
    text=re.sub(r'\'','',text) #remove apici
    text=re.sub(r'\n','',text) #remove spazio vuoto

    return text

In [16]:
len(df['hashtags'])

28381

In [17]:
df['clean_tags']=df['hashtags'].apply(lambda x: remove_content(x))
len(df['clean_tags'])

28381

In [18]:
df.insert(0, 'clean_tags', df.pop('clean_tags'))

In [19]:
df.insert(1, 'hashtags', df.pop('hashtags'))

In [20]:
df.head(1)

,clean_tags,hashtags,date,user_id,username,name,tweet,language,mentions,urls,replies_count,retweets_count,likes_count,link,retweet,reply_to
5,"fossilfuels, wind, solar, green, hydrogen, ene...","['fossilfuels', 'wind', 'solar', 'green', 'hyd...",2022-07-10,59386332,arikring,Arik Ring - Energy Engineering Expert,@ProfStrachan @ECOWARRIORSS @ChristineMilne @m...,en,"[{'screen_name': 'guytaur', 'name': 'scott. tr...",['https://reneweconomy.com.au/integrated-wind-...,2,54,59,https://twitter.com/arikring/status/1546251358...,False,"[{'screen_name': 'ProfStrachan', 'name': 'Prof..."


In [21]:
#Salvo le modifiche nel df nuovo
df.to_csv('../Dataset FFF/fff.csv', index=False)

###  Creazione network di partenza

In [22]:
df1=pd.read_csv("../Dataset FFF/fff.csv")
df1.head(1)

,clean_tags,hashtags,date,user_id,username,name,tweet,language,mentions,urls,replies_count,retweets_count,likes_count,link,retweet,reply_to
0,"fossilfuels, wind, solar, green, hydrogen, ene...","['fossilfuels', 'wind', 'solar', 'green', 'hyd...",2022-07-10,59386332,arikring,Arik Ring - Energy Engineering Expert,@ProfStrachan @ECOWARRIORSS @ChristineMilne @m...,en,"[{'screen_name': 'guytaur', 'name': 'scott. tr...",['https://reneweconomy.com.au/integrated-wind-...,2,54,59,https://twitter.com/arikring/status/1546251358...,False,"[{'screen_name': 'ProfStrachan', 'name': 'Prof..."


In [23]:
#Ti crea un dizionario in cui le chiavi sono i tag singoli e i valori sono gli id degli user
tags_dict = {}
for i in range(len(df1)):
    ll = str(df1["clean_tags"][i]).split(",")[:-1]
    for tg in ll:
        if tg not in tags_dict:
            tags_dict[tg] = [df1["user_id"][i]]
        else:
            tags_dict[tg].append(df1["user_id"][i])

In [24]:
#Crea la lista di tags dalle chiavi del dizionario e li associa alla variabile tag
tags = list(tags_dict.keys())
#crea una lista costituita da coppie di tags
couple_tags = []
for i in range(len(tags)):
    for j in tags[i+1:]:
        couple_tags.append((tags[i], j)) 

In [25]:
couple_tags

[('fossilfuels', ' wind'),
 ('fossilfuels', ' solar'),
 ('fossilfuels', ' green'),
 ('fossilfuels', ' hydrogen'),
 ('fossilfuels', ' energy'),
 ('fossilfuels', ' climateaction'),
 ('fossilfuels', ' renewableenergy'),
 ('fossilfuels', ' carbon'),
 ('fossilfuels', ' demvoice1'),
 ('fossilfuels', ' sdgs'),
 ('fossilfuels', ' climatecrisis'),
 ('fossilfuels', 'klimakrise'),
 ('fossilfuels', ' govegan'),
 ('fossilfuels', 'fridaysforfuture'),
 ('fossilfuels', ' riseupmovement'),
 ('fossilfuels', 'notmytaxonomy'),
 ('fossilfuels', 'gocarfree'),
 ('fossilfuels', ' istayontheground'),
 ('fossilfuels', ' flightfree'),
 ('fossilfuels', ' eatonlyplants'),
 ('fossilfuels', ' shopstop'),
 ('fossilfuels', ' fridaysforfuture'),
 ('fossilfuels', 'climatecrisis'),
 ('fossilfuels', ' climatestrike'),
 ('fossilfuels', ' climate'),
 ('fossilfuels', ' savesoilbengaluru'),
 ('fossilfuels', ' gaskrise'),
 ('fossilfuels', ' grünen'),
 ('fossilfuels', 'peoplepower'),
 ('fossilfuels', ' actnow'),
 ('fossilfuels'

In [26]:
len(couple_tags)

62401206

In [27]:
len(tags)

11172

In [28]:
len(tags)

11172

In [29]:
source = []
target = []
count = []
nodes = []
for i in couple_tags:
    ll = list(set(tags_dict[i[0]]) & set(tags_dict[i[1]]))
    if len(ll)>0:
        source.append(i[0])
        target.append(i[1])
        count.append(len(ll))
        nodes.append(i[0])
        nodes.append(i[1])

In [30]:
nodes = list(set(nodes)) #list()crea una lista; set(nodes) prende i nodi senza ripetizioni

In [31]:
len(nodes)

10977

In [32]:
nodes

[' todoslossantos',
 'ggultras',
 ' ees20',
 'queenelizabeth',
 ' massextinction',
 ' cancún',
 ' workshop',
 ' lichtaus',
 ' wetsuwetensolidarity',
 ' ilfordfp5',
 ' razetozero',
 ' oil',
 ' davos2020',
 ' kemmerich',
 ' endcoal',
 ' katebernadette',
 'kesselbambule',
 ' gruene',
 'autoprämie',
 'pollutersout',
 ' futures',
 ' immigration',
 'coronapandemie',
 ' stayathome',
 ' indymedia',
 ' hüpfdeutsche',
 'neckarwestheim',
 ' scioperoperilclima',
 ' gegendiea49',
 'lausanne',
 ' ambiente',
 'iklimiçinokulgrevi',
 'union',
 ' ge2019',
 'elmshorn',
 ' bs2911',
 ' cambiamenticlimatici',
 ' linken',
 'transpride',
 'unteilbar',
 ' europawahl',
 'mumbai',
 ' xmas21',
 ' ratifyparis',
 ' brasilien',
 'climatereporting',
 ' plasticstreaty',
 'katyayan',
 ' wastetoenergy',
 ' wo0606',
 ' cleanup',
 ' tiktokdance',
 'fossilfriday',
 'globalstrike',
 'filme',
 'polizeigewalt',
 ' riberasacra',
 'stopp',
 ' connewitz',
 'strongertogether',
 ' jusos',
 ' kxlisdead',
 ' yourself',
 ' fightforev

In [33]:
new_df = pd.DataFrame({"Id": nodes})
new_df.to_csv("../Dataset FFF/nodes.csv", index=False)

In [34]:
new_df = pd.DataFrame({"Source": source,
                       "Target": target,
                      "Count": count})
new_df.to_csv("../Dataset FFF/edges.csv", index=False)

In [45]:
others_df = pd.DataFrame({"Id": nodes})
others_df.to_csv("../Dataset FFF/nodes.csv", index=False)

In [46]:
others_df = pd.DataFrame({"Source": source,
                       "Target": target,
                      "Count": count})
others_df.to_csv("../Dataset FFF/edges.csv", index=False)

### Creazione dataset da 10k e 3k nodi

###### Controllo preliminare dei nodi ed edges


In [35]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import math
import numpy as np
import networkx as nx
from operator import itemgetter

In [36]:
path = "../Dataset FFF/edges.csv"
path_all = "../Dataset FFF/fff.csv"
path_nodes = "../Dataset FFF/nodes.csv"

In [37]:
fff = pd.read_csv(path)

In [38]:
fff.sort_values('Count', ascending=False)

,Source,Target,Count
11294,fridaysforfuture,fridaysforfuture,679
17765,fridaysforfuture,climatestrike,430
1945,climateaction,fridaysforfuture,312
17798,fridaysforfuture,climatestrike,310
6599,climatecrisis,fridaysforfuture,301
...,...,...,...
129597,fridaymotivation,lifestyle,1
129596,fridaymotivation,life,1
129595,fridaymotivation,speciesism,1
129594,fridaymotivation,animalliberation,1


In [39]:
G = nx.read_edgelist(path, comments = "S", delimiter = ",", data=(("Count", int),))

In [40]:
degree_dict = dict(G.degree(G.nodes()))
nx.set_node_attributes(G, degree_dict, 'degree')
sorted_degree = sorted(degree_dict.items(), key = itemgetter(1), reverse = True)
print(" ---- Top 20 nodes by degree ----")
for d in sorted_degree[:20]:
    print("\t" + str(d))

 ---- Top 20 nodes by degree ----
	('fridaysforfuture', 9477)
	('climatestrike', 4041)
	('climatecrisis', 3410)
	('climateaction', 3361)
	('climateemergency', 2982)
	('climatejustice', 2593)
	('climatestrikeonline', 2503)
	('climatechange', 2447)
	('klimastreik', 2138)
	('peoplenotprofit', 2051)
	('klimakrise', 2047)
	('klimaschutz', 2032)
	('climateactionnow', 2027)
	('nomoreemptypromises', 2024)
	('uprootthesystem', 1924)
	('cop26', 1795)
	('climate', 1791)
	('fightfor1point5', 1745)
	('gretathunberg', 1716)
	('fff', 1684)


In [41]:
nodes = []
others = []
for i in range(len(sorted_degree)):
    if i<10000:
        nodes.append(sorted_degree[i][0])
        #print("N.nodi:",nodes)
for i in range(len(sorted_degree)):
    if i<3000:
        others.append(sorted_degree[i][0])

##### Abbiamo 15k nodi

In [42]:
len(nodes)

10000

In [43]:
new_fff = fff[(fff["Source"].isin(nodes)) & (fff["Target"].isin(nodes))] 

In [47]:
new_df.to_csv("../Dataset FFF/edges_10K.csv",  index=False)
others_df.to_csv("../Dataset FFF/edges_3K.csv",  index=False)

In [48]:
new_df = pd.DataFrame({"Id": nodes})
new_df.to_csv("../Dataset FFF/nodes_10K.csv", index=False)

In [49]:
others_df = pd.DataFrame({"Id": others})
others_df.to_csv("../Dataset FFF/nodes_3K.csv", index=False)

In [50]:
edges=pd.read_csv('../Dataset FFF/edges_10k.csv')

## Risultato pulizia:

In [51]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 15169
Number of edges: 300987
Average degree:  39.6845


In [52]:
new_df

,Id
0,fridaysforfuture
1,climatestrike
2,climatecrisis
3,climateaction
4,climateemergency
...,...
9995,mietwucher
9996,ldk22nrw
9997,happyearthday2022
9998,smallworldafterall


In [54]:
others_df

,Id
0,fridaysforfuture
1,climatestrike
2,climatecrisis
3,climateaction
4,climateemergency
...,...
2995,meatloaf
2996,emergenzaclimatica
2997,aktivisten
2998,happynewyear
